In [1]:
import os
import sys
from PIL import Image
import numpy as np
import cv2
current_path = os.path.abspath(os.getcwd())


In [6]:
TL_Label =  (250,170,30, 255)

In [7]:
depth_path = os.path.join(current_path, "Depth")
depth_deb_path = os.path.join(current_path, "DepthDebug")
gt_deb_path = os.path.join(current_path, "GTDebug")
rgb_path = os.path.join(current_path, "RGB")

In [61]:
mask = Image.open(current_path + "/example_semantic_93.png") 
pixels = np.array(mask)
pixels_mask = np.array([int(pixels[i][j][0]==TL_Label[0] and pixels[i][j][1]==TL_Label[1]) for i in range(len(pixels)) for j in range(len(pixels[0]))])

In [62]:
pixels_mask = np.reshape(pixels_mask,(1080,1920))
pixels_mask = [pixels_mask, pixels_mask, pixels_mask, pixels_mask]
pixels_mask = np.swapaxes(pixels_mask, 0,2)
pixels_mask = np.swapaxes(pixels_mask, 1,0)
inv_mask = 1-pixels_mask

In [63]:
depth = Image.open(current_path + "/example_depth_93.png") 
depth_pixels = np.array(depth)

# Getting depth for traffic lights only
new_depth_pixels = depth_pixels*pixels_mask
new_depth = Image.fromarray(new_depth_pixels.astype(np.uint8)).save(current_path+"/test_depth.png")

In [66]:
import torch

In [67]:
model = torch.hub.load('pytorch/vision:v0.9.0', 'deeplabv3_resnet101', pretrained=True)


Downloading: "https://github.com/pytorch/vision/archive/v0.9.0.zip" to /home/ghadeer/.cache/torch/hub/v0.9.0.zip
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /home/ghadeer/.cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth
100.0%


In [68]:
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [71]:
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')
    print("Using CUDA")


Using CUDA


In [74]:
print(input_batch.shape)

torch.Size([1, 3, 1213, 1546])


In [70]:
with torch.no_grad():
    output = model(input_batch)['out'][0]
output_predictions = output.argmax(0)

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 256, 1, 1])

In [106]:
a = np.array([1, 2, 3, 4, 5, 6])
b = a + 5

In [107]:
a* (255,255,255,255)

ValueError: operands could not be broadcast together with shapes (6,) (4,) 

In [15]:
print(pixels2[0][1000:1010])
print(pixels2[1][1000:1010])
print(pixels2[2][1000:1010])

[305 305 305 305 305 305 305 305 305 305]
[1135 1136 1137 1138 1139 1140 1141 1142 1143 1144]
[1 1 1 1 1 1 1 1 1 1]


In [43]:
a1 = [i for i in range(len(pixels_mask)) if pixels_mask[i]==True]

7156